# 5. Pour le fun, créer une API

In [36]:
#tester le fichier pickle
import pickle
# Chargez votre modèle préenregistré avec Pickle
with open('modele.pickle', 'rb') as model_file:
    model = pickle.load(model_file)
    
with open('sc_fit.pickle', 'rb') as model_file:
    sc_fit = pickle.load(model_file)

nb_immeubles = 50000000000
pct_charge = 100

pct_distrib_2018 = 5
pct_distrib_2019 = 4.5
pct_distrib_2020 = 4
pct_distrib_2021 = 5

capital = 0

 ##################### ##################### SCPI_plus_value = 0
SCPI_rendement = 1


type_SCPI = 0

categorie_act_log = 0
categorie_bureaux = 0
categorie_commerce = 1
categorie_diversifiee = 0
categorie_residentiel = 0
categorie_sante_education = 0
categorie_tourisme_loisirs = 0    


variation_prix_2019 = pct_distrib_2019-pct_distrib_2018
variation_prix_2020 = pct_distrib_2020-pct_distrib_2019
variation_prix_2021 = pct_distrib_2021-pct_distrib_2020
variation_prix_2018 = (variation_prix_2019+variation_prix_2020+variation_prix_2021)/3


#X = [[capital, nb_immeubles, pct_charge, variation_prix_2018, variation_prix_2019,
#                              variation_prix_2020, variation_prix_2021, pct_distrib_2018,
#                              pct_distrib_2019, pct_distrib_2020, pct_distrib_2021, SCPI_plus_value,
#                             SCPI_rendement,categorie_act_log,categorie_bureaux,categorie_commerce,
#                             categorie_diversifiee,categorie_residentiel,categorie_sante_education,categorie_tourisme_loisirs]]

X = [[type_SCPI, capital, nb_immeubles, pct_charge, variation_prix_2018, variation_prix_2019,
                              variation_prix_2020, variation_prix_2021, pct_distrib_2018,
                              pct_distrib_2019, pct_distrib_2020, pct_distrib_2021, categorie_act_log,categorie_bureaux,categorie_commerce,
                             categorie_diversifiee,categorie_residentiel,categorie_sante_education,categorie_tourisme_loisirs]]
X_sc = sc_fit.transform(X)
prediction = model.predict(X_sc)
print(X_sc)
print(prediction)
#en testant, le nombre d'immeuble, le pct de charge et le type de SCPI ne change pas grand chose, on va alors simplifier l'API par la suite

[[-5.24404424e+00 -1.93649167e+00  4.77230042e+08  3.13120028e+00
  -4.27516810e-01 -4.83523404e-01 -7.27538733e-01  5.08241955e-01
   5.15638652e-01 -1.61724593e-02 -1.07575162e-01  5.46751439e-01
  -1.33630621e-01 -9.48683298e-01  1.93649167e+00 -4.04061018e-01
  -2.74721128e-01 -1.90692518e-01 -2.35702260e-01]]
[5.22232833]


C:\Users\paulj\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [37]:
from flask import Flask, render_template, request, jsonify
import pickle

app = Flask(__name__)

# Chargez votre modèle préenregistré avec Pickle
with open('modele.pickle', 'rb') as model_file:
    model = pickle.load(model_file)
    
with open('sc_fit.pickle', 'rb') as model_file:
    sc_fit = pickle.load(model_file)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    # Récupérez les variables d'entrée depuis le formulaire

    nb_immeubles = 130
    pct_charge = 30

    #pct_distrib_2018 = float(request.form['pct_distrib_2018'])
    pct_distrib_2018 = float(request.form['pct_distrib_2018'])
    pct_distrib_2019 = float(request.form['pct_distrib_2019'])
    pct_distrib_2020 = float(request.form['pct_distrib_2020'])
    pct_distrib_2021 = float(request.form['pct_distrib_2021'])
    


    variation_prix_2019 = pct_distrib_2019-pct_distrib_2018
    variation_prix_2020 = pct_distrib_2020-pct_distrib_2019
    variation_prix_2021 = pct_distrib_2021-pct_distrib_2020
    variation_prix_2018 = (variation_prix_2019+variation_prix_2020+variation_prix_2021)/3


    capital_var = request.form['capital']    

    categorie_var = request.form['categorie']

    capital = 0
    
    type_SCPI = 0
    
    categorie_act_log = 0
    categorie_bureaux = 0
    categorie_commerce = 0
    categorie_diversifiee = 0
    categorie_residentiel = 0
    categorie_sante_education = 0
    categorie_tourisme_loisirs = 0

    if capital_var == 'Fixe':
        capital = 1
  
        
    if categorie_var == 'Activités / Logistique':
        categorie_act_log = 1
    elif categorie_var == 'Bureaux':
        categorie_bureaux = 1
    elif categorie_var == 'Commerce':
        categorie_commerce = 1
    elif categorie_var == 'Diversifiée':
        categorie_diversifiee = 1
    elif categorie_var == 'Résidentiel':
        categorie_residentiel = 1
    elif categorie_var == 'Santé et éducation':
        categorie_sante_education = 1
    elif categorie_var == 'Tourisme et loisirs':
        categorie_tourisme_loisirs = 1
    else:
        categorie_diversifiee = 1
    

    # Effectuez la prédiction avec votre modèle
    
    
    X = [[type_SCPI, capital, nb_immeubles, pct_charge, variation_prix_2018, variation_prix_2019,
                              variation_prix_2020, variation_prix_2021, pct_distrib_2018,
                              pct_distrib_2019, pct_distrib_2020, pct_distrib_2021, categorie_act_log,categorie_bureaux,categorie_commerce,
                             categorie_diversifiee,categorie_residentiel,categorie_sante_education,categorie_tourisme_loisirs]]
    
    X_sc = sc_fit.transform(X)
    prediction = model.predict(X_sc)

    # Retournez la prédiction en tant que réponse JSON
    return render_template('index.html', prediction=round(prediction[0], 2))

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\paulj\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
#exécuter dans un jupyter notebook :
!python app.py

#http://localhost:5000/

^C
